# COVID -19 MASS GOV

In [1]:
#start session
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2')
#from pyspark import SparkContext
#from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import SparkSession
from timeit import default_timer as timer
#from modules import claims_sql
from pyspark.sql.functions import col, round

import sys
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
#DW Netezza Login Information
import getpass
import json
from datetime import date, datetime
import pandas as pd
pd.set_option('display.max_columns', None,'display.max_columns', None)

from IPython.core.interactiveshell import InteractiveShell #display results setting
InteractiveShell.ast_node_interactivity = "all"

# uid = input("Enter LAN ID: ")
# pwd = getpass.getpass("Enter LAN pwd: ")

In [2]:
spark = SparkSession \
        .builder \
        .appName("COVID-19_MassGov") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.memory", "48G") \
        .config("spark.executor.cores", "5") \
        .getOrCreate()

## COVID-19 MA cases everyday

In [62]:
#download All data from mass.gov
#https://stackoverflow.com/questions/48800385/how-to-download-ms-word-docx-file-in-python-with-raw-data-from-http-url/48811118
import requests

dates = ['3-20','3-21']
dates = ['3-21']
months = ['january','february','march','april','may','june',\
          'july','august','september','october','november','december']

def download_covid_daily_data(dt):
    mth = months[int(dt.split('-')[0])-1]
    dy = dt.split('-')[1]
    f_url = 'https://www.mass.gov/doc/covid-19-cases-in-massachusetts-as-of-'+mth+'-'+str(dy)+'-2020-accessible/download/'
    f_name = 'covid-19-case-report-'+str(dt)+'-2020-acc.docx'
    print(f_url,f_name)
    
    def save_link(book_link, book_name):
        the_book = requests.get(book_link, stream=True)
        with open(book_name, 'wb') as f:
            for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
                f.write(chunk)

    save_link(f_url,f_name)
    
for day in dates:
    download_covid_daily_data(day)

https://www.mass.gov/doc/covid-19-cases-in-massachusetts-as-of-march-21-2020-accessible/download/ covid-19-case-report-3-21-2020-acc.docx


In [45]:
#https://stackoverflow.com/questions/10366596/how-to-read-contents-of-an-table-in-ms-word-file-using-python

In [4]:
uid = 'abaner02'
sql = 'select * from analytics.w5_covid_19_MASSGOV_'+uid
df = spark.sql(sql).toPandas()
df

Barnstable  Berkshire  Bristol  Dukes  Essex  Franklin  Hampden  Hampshire  \
0          24         23       24      1     60         2       12          4   
1          11         21       14      1     41         2        9          2   
2           5         18        6      0     19         1        3          1   
3           9         20        6      0     29         1        3          2   
4           2         17        5      0     14         1        2          0   
5           2         14        5      0      8         0        1          0   
6           1         11        2      0      8         0        1          0   
7           0          9        0      0      2         0        0          0   
8           1          9        1      0      5         0        0          0   
9           1          9        1      0      6         0        1          0   

   Middlesex  Norfolk  Plymouth  Suffolk  Worcester  Unknown  Total  deaths  \
0        199       75        25      126         37       34    646       5   
1        177       69        20      108         24       26    525       1   
2        119       52         5       72         14       13    328       0   
3        144       64        11       86         19       19    413       1   
4        100       45         5       51         10        4    256       0   
5         89       43         5       42          8        1    218       0   
6         83       36         3       36          6       10    197       0   
7         60       24         0       26          2        0    123       0   
8         65       28         0       27          2        0    138       0   
9         75       31         1       31          6        2    164       0   

   gender_Female  gender_Male  hosp_Patientwashospitalized  \
0            308          338                           71   
1            249          275                           61   
2            159          169                           43   
3            195          218                           58   
4            119          137                           27   
5            102          116                           21   
6             93          104                           14   
7             54           69                           10   
8             64           74                           11   
9             74           90                           13   

   hosp_Patientwasnothospitalized  hosp_UnderInvestigation  \
0                             263                      312   
1                             215                      249   
2                             160                      125   
3                             199                      156   
4                             151                       78   
5                             145                       52   
6                             123                       60   
7                             101                       12   
8                             105                       22   
9                             115                       36   

   exp_Biogenemployeescontacts  exp_Travelrelated  \
0                            0                 68   
1                            0                 53   
2                            0                 34   
3                            0                 49   
4                            0                 26   
5                            0                 24   
6                            0                 18   
7                           94                  5   
8                            0                  5   
9                            0                 13   

   exp_BerkshireMedicalCentercontacts  exp_Underinvestigation  \
0                                   0                     396   
1                                   0                     306   
2                                   0                     151   
3                              

In [5]:
import zipfile
import xml.etree.ElementTree

WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'
TABLE = WORD_NAMESPACE + 'tbl'
ROW = WORD_NAMESPACE + 'tr'
CELL = WORD_NAMESPACE + 'tc'

def read_daily_covid_data(dt):
    data_str = ''
    file_x = 'covid-19-case-report-'+dt+'-2020.docx'
    print(file_x)
    with zipfile.ZipFile(file_x) as docx:
        tree = xml.etree.ElementTree.XML(docx.read('word/document.xml'))
    
      
    for table in tree.iter(TABLE):
        for row in table.iter(ROW):
            for cell in row.iter(CELL):
                data_str = data_str + ' '.join(node.text for node in cell.iter(TEXT)) +'\n'
                #print(data_str)
                #print(type(data_str))
    return data_str
                
data = []
#dates = ['3-13','3-14','3-15','3-16','3-17','3-18','3-19','3-20','3-21','3-22','3-23']
dates = ['3-23','3-24']
data_dict_dt = {}
for day in dates:
    data_dict_dt[day] = {}
    curr = read_daily_covid_data(day)
    #print('County --->' , curr.split('County')[1].split('Sex')[0])
    p1 = curr.split('County')
   
    county_raw = [ m.replace(' ','') for m in p1[1].split('Sex')[0].split('\n') if m != '']
    counties = [i[1] for i in enumerate(county_raw) if i[0]%2 == 0]
    counties_data = [i[1] for i in enumerate(county_raw) if i[0]%2 != 0]
    county = dict(list(zip(counties,counties_data)))
    
    gender_raw = p1[1].split('Sex')[1].split('Exposure')[0]
    if gender_raw.find('Age Group') == -1 :
        gender_i = [ m.replace(' ','') for m in gender_raw.split('\n') if m != '']
        age = {}
    else:
        gender_i = [ m.replace(' ','') for m in gender_raw.split('Age Group')[0].split('\n') if m != '']
        # Add Agegroup
        ageraw =gender_raw.split('Age Group')[1]
        ageraw = [ m.replace(' ','') for m in ageraw.split('\n') if m != '']
        age_i = [i[1] for i in enumerate(ageraw) if i[0]%2 == 0]
        age_data = [i[1] for i in enumerate(ageraw) if i[0]%2 != 0]
        age = dict(list(zip(age_i,age_data)))
        
    
    g_i = [i[1] for i in enumerate(gender_i) if i[0]%2 == 0]
    g_data = [i[1] for i in enumerate(gender_i) if i[0]%2 != 0]
    gender = dict(list(zip(g_i,g_data)))
        
     
    p2 = p1[1].split('Sex')[1].split('Exposure')[1]
    
    exposure_raw = p2.split('Hospitalization')[0]
    #print(exposure_raw.find('Deaths'),exposure_raw)
    if exposure_raw.find('Deaths') == -1 :
        x_raw = [ m.replace(' ','') for m in exposure_raw.split('\n') if m != '']
        deaths =0
    else:
        x_raw  = [ m.replace(' ','') for m in exposure_raw.split('Deaths')[0].split('\n') if m != '']
        deaths = int([ m.replace(' ','') for m in exposure_raw.split('Deaths')[1].split('\n') if m != ''][1])
        
    x_i = [i[1] for i in enumerate(x_raw) if i[0]%2 == 0]
    x_data = [i[1] for i in enumerate(x_raw) if i[0]%2 != 0]
    exposure = list(zip(x_i,x_data))
    
    hospitalizations_raw = p2.split('Hospitalization')[1].replace(' ','')
    hospitalizations_raw2 = hospitalizations_raw.split('MAStatePublicHealthLaboratory')[0]
    #print(hospitalizations_raw2)
    h_raw = [ m for m in hospitalizations_raw2.split('\n') if m != '']
    h_i = [i[1] for i in enumerate(h_raw) if i[0]%2 == 0]
    h_data = [i[1] for i in enumerate(h_raw) if i[0]%2 != 0]
    hospitalizations = dict(list(zip(h_i,h_data)))
    
    try:
        p3 = hospitalizations_raw.split('MAStatePublicHealthLaboratory')[1]
        #print('9'* 9)
        ma_lab = p3.split('Laboratory')[0].replace('\n','|')
        ma_lab = ma_lab.split('Patients')[1].split('|')
        ma_lab = [m.replace(' ','') for m in ma_lab if m != '']
        
        lab = p3.split('Laboratory')[1].replace('\n','|')
        lab = lab.split('TotalTested')[1].replace('*','').split('|')
        lab =[m.replace(' ','') for m in lab if m != '']
        #labs = [i[1] for i in enumerate(lab) if i[0]%3 == 0]
        lab = [(i[1],lab[i[0]+1].replace(' ',''),lab[i[0]+2].replace(' ','')) for i in enumerate(lab) if i[0]%3 == 0]
        lab = [('MAStateLab',ma_lab[0],ma_lab[1])] +lab
        lab_dict = {}
        for m in lab:
            lab_dict[m[0]] = {}
            lab_dict[m[0]]['tested_positive'] = m[1]
            lab_dict[m[0]]['tested_total'] = m[2]
        
        lab = lab_dict
    except:
        #print('6'* 9)
        #ma_lab = []
        lab = []
    
    data.append([day,county,gender,exposure,hospitalizations,ma_lab,lab,deaths])
    data_dict_dt[day]['county'] = county
    data_dict_dt[day]['gender'] = gender
    data_dict_dt[day]['exposure'] = exposure
    data_dict_dt[day]['hospitalizations'] = hospitalizations
    data_dict_dt[day]['age'] = age
    data_dict_dt[day]['lab'] = lab
    data_dict_dt[day]['deaths'] = deaths
    
#County data
county_dict = {}
counties = list(data[len(data)-1][1].keys())
for m in counties:
    if m.find('Dukes') != -1:
        m1 = 'Dukes'
    else:
        m1=m
    county_dict[m1] = {}   
    for day,data_y in data_dict_dt.items():
        try:
            county_dict[m1][day] = int(data_y['county'][m])
        except:
            county_dict[m1][day] = 0
time_data = pd.DataFrame(county_dict)
#time_data['Total'] = time_data.sum()
time_data.loc[:,'Total'] = time_data.sum(numeric_only=True, axis=1)

# Deaths
death_d = {}
death_d['deaths'] = {} 
for day,data_x in data_dict_dt.items():
    #print(day,data_x)
    death_d['deaths'][day] = data_x['deaths']
df_death = pd.DataFrame(death_d)

#gender
last_i = list(data_dict_dt.keys())[-1]
gen_list = list(data_dict_dt[last_i]['gender'].keys())
print(gen_list)
g_dict = {}
for m in gen_list:
    g_dict['gender_'+m] = {}
    for day,data_y in data_dict_dt.items():
        try:
            g_dict['gender_'+m][day] = int(data_y['gender'][m])
        except:
            g_dict['gender_'+m][day] = 0
            
g_data = pd.DataFrame(g_dict)

#age
age_list = list(data_dict_dt[last_i]['age'].keys())
print(age_list)
age_dict = {}
for m in age_list:
    m1 = m.replace('yearsofage','').replace('≤','lessthan').replace('≥','gtthan')
    age_dict['age_'+m1] = {}
    for day,data_y in data_dict_dt.items():
        try:
            age_dict['age_'+m1][day] = int(data_y['age'][m])
        except:
            age_dict['age_'+m1][day] = 0
            
age_data = pd.DataFrame(age_dict)

#hospitalizations
hosp_list = list(data_dict_dt[last_i]['hospitalizations'].keys())
print(gen_list)
h_dict = {}
for m in hosp_list:
    h_dict['hosp_'+m] = {}
    for day,data_y in data_dict_dt.items():
        try:
            h_dict['hosp_'+m][day] = int(data_y['hospitalizations'][m])
        except:
            h_dict['hosp_'+m][day] = 0
            
h_data = pd.DataFrame(h_dict)

#lab testin
print(data_dict_dt[last_i]['lab'])
lab_list = list(data_dict_dt[last_i]['lab'].keys())
print(lab_list)
l_dict = {}
for m in lab_list:
    l_dict['test_'+m+'_positive'] = {}
    l_dict['test_'+m+'_total'] = {}
    for day,data_y in data_dict_dt.items():
        try:
            l_dict['test_'+m+'_positive'][day] = data_y['lab'][m]['tested_positive']
            l_dict['test_'+m+'_total'][day] = data_y['lab'][m]['tested_total']
        except:
            l_dict['test_'+m+'_positive'][day] = 0
            l_dict['test_'+m+'_total'][day] = 0
            
l_data = pd.DataFrame(l_dict)

#exposure
exp_list = []
x_dict = {}
for day,data_y in data_dict_dt.items():
    #print(day,data_y['exposure'])
    for m in data_y['exposure']:
        if m[0] not in exp_list:
            exp_list.append(m[0])
#print(exp_list)
for m in exp_list:
    x_dict['exp_'+m.replace('/','').replace('*','')] = {}
    for day,data_y in data_dict_dt.items():
        #print(data_y['exposure'])
        x_dict['exp_'+m.replace('/','').replace('*','')][day] = 0
        for exp in data_y['exposure']:
            #print(m, exp[0],m == exp[0])
            if m == exp[0]:
                #print('HERE',exp[1])
                x_dict['exp_'+m.replace('/','').replace('*','')][day] = int(exp[1])
#print(x_dict)
x_data = pd.DataFrame(x_dict)
biogen_cols =[ m for m in x_data if m.find('Biogen') > -1]
biogen_df = x_data[biogen_cols]
biogen_df['exp_Biogen_Total'] = biogen_df.sum(numeric_only=True, axis=1)
#print(biogen_cols)
    
time_data = time_data.join(df_death).join(g_data)\
        .join(h_data).join(x_data).join(biogen_df.loc[:,'exp_Biogen_Total']).join(l_data).join(age_data)
#time_data
print(time_data)
#time_data.to_csv('covid_19_mass_gov_time_series.csv')
data_dict_dt['3-19']['county']

covid-19-case-report-3-23-2020.docx
covid-19-case-report-3-24-2020.docx
['Female', 'Male', 'Unknown']
['≤19yearsofage', '20-29yearsofage', '30-39yearsofage', '40-49yearsofage', '50-59yearsofage', '60-69yearsofage', '≥70yearsofage', 'Unknown']
['Female', 'Male', 'Unknown']
[]


AttributeError: 'list' object has no attribute 'keys'

In [104]:
#time_data.loc[:,[counties]]
counties_dukes = [m if m.find('Dukes') == -1 else 'Dukes' for m in counties ]
counties_dukes_Transposed = time_data.loc[:,counties_dukes].T
counties_dukes_Transposed.to_csv('county_lvl_covid_19_mass_gov_time_series.csv')
counties_dukes

['Barnstable',
 'Berkshire',
 'Bristol',
 'Dukes',
 'Essex',
 'Franklin',
 'Hampden',
 'Hampshire',
 'Middlesex',
 'Norfolk',
 'Plymouth',
 'Suffolk',
 'Worcester',
 'Unknown']

In [96]:
time_data['test_MAStateLab_positive'] = time_data['test_MAStateLab_positive'].astype('str')
time_data['test_MAStateLab_total'] = time_data['test_MAStateLab_total'].astype('str')
time_data['test_LabCorp_positive'] = time_data['test_LabCorp_positive'].astype('str')
time_data['test_LabCorp_total'] = time_data['test_LabCorp_total'].astype('str')
time_data['test_QuestDiagnostics_positive'] = time_data['test_QuestDiagnostics_positive'].astype('str')
time_data['test_QuestDiagnostics_total'] = time_data['test_QuestDiagnostics_total'].astype('str')
time_data['test_Otherlaboratories_positive'] = time_data['test_Otherlaboratories_positive'].astype('str')
time_data['test_Otherlaboratories_total'] = time_data['test_Otherlaboratories_total'].astype('str')
time_data = time_data.reset_index()
time_data['day'] = time_data['index']
del time_data['index']
print(time_data)
df_mass = spark.createDataFrame(time_data.fillna('0'))

#df_mass.write.mode("overwrite").format('parquet').saveAsTable('analytics.w5_covid_19_MASSGOV_'+uid)

   Barnstable  Berkshire  Bristol  Dukes  Essex  Franklin  Hampden  Hampshire  \
0           0          9        0      0      2         0        0          0   
1           1          9        1      0      5         0        0          0   
2           1          9        1      0      6         0        1          0   
3           1         11        2      0      8         0        1          0   
4           2         14        5      0      8         0        1          0   
5           2         17        5      0     14         1        2          0   
6           5         18        6      0     19         1        3          1   
7           9         20        6      0     29         1        3          2   
8          11         21       14      1     41         2        9          2   
9          24         23       24      1     60         2       12          4   

   Middlesex  Norfolk  Plymouth  Suffolk  Worcester  Unknown  Total  deaths  \
0         60       24        